In [22]:
# !!pip uninstall tensorflow, mediapipe
# !pip install tensorflow==2.20.0
# !pip install mediapipe
!pip show mediapipe

Name: mediapipe
Version: 0.10.21
Summary: MediaPipe is the simplest way for researchers and developers to build world-class ML solutions and applications for mobile, edge, cloud and the web.
Home-page: https://github.com/google/mediapipe
Author: The MediaPipe Authors
Author-email: mediapipe@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: absl-py, attrs, flatbuffers, jax, jaxlib, matplotlib, numpy, opencv-contrib-python, protobuf, sentencepiece, sounddevice
Required-by: 


In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, TimeDistributed # type: ignore
from tensorflow.keras.optimizers import Adam #type: ignore
from scipy.io import loadmat  # Read file .mat
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import glob
import logging
import mediapipe as mp
from tqdm import tqdm

In [24]:
# Model initialization
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True)

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()

In [25]:
# ====== 1. Face Mask Creation ======
def create_face_mask_with_colors(image):
    """
    Detects a face in an image using MediaPipe Face Mesh, creates a convex hull mask
    around the face landmarks, and returns a new image where everything
    outside the face mask is blacked out.
    """
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_image)

    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            points = np.array([(int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0]))
                               for landmark in face_landmarks.landmark], dtype=np.int32)

            hull = cv2.convexHull(points)

            cv2.fillPoly(mask, [hull], 255)

    masked_face = cv2.bitwise_and(image, image, mask=mask)
    return masked_face

In [26]:
# ====== 2. Skin Segmentation ======
def skin_segmentation(face_mask):
    """
    This function converts the BGR `face_mask` to the HSV color space
    to isolate skin-tone pixels base odn a predefined range of
    Hue, Saturation, and Value. It filters out non-skin pixels
    like eyes, eyebrows, and mouth.
    """
    hsv = cv2.cvtColor(face_mask, cv2.COLOR_BGR2HSV)

    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)

    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
    return cv2.bitwise_and(face_mask, face_mask, mask=skin_mask)

In [27]:
# ====== 3. Signal Processing Functions ======
def blackout_outside_dynamic_threshold(frame, lower_factor=0.48, upper_factor=1.74):
    """Filters out pixels that are too dark or too bright based on a dynamic threshold."""
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    non_zero_pixels = gray_image[gray_image != 0]

    average_value = np.median(non_zero_pixels) if non_zero_pixels.size > 0 else 0

    lower_threshold = max(0, int(average_value * lower_factor))
    upper_threshold = min(255, int(average_value * upper_factor))

    mask = (gray_image >= lower_threshold) & (gray_image <= upper_threshold)

    updated_frame = np.zeros_like(frame)
    updated_frame[mask] = frame[mask]

    return updated_frame

In [28]:
# ====== 4. RGB Signal Extraction ======
def extract_rgb_signals(frame):
    """Extracts mean RGB values from the given frame."""
    face_pixels = frame[frame.sum(axis=2) > 0]

    if len(face_pixels) == 0:
        logger.debug("No face pixels detected")
        return (0, 0, 0)

    mean_r = int(np.mean(face_pixels[:, 2]))
    mean_g = int(np.mean(face_pixels[:, 1]))
    mean_b = int(np.mean(face_pixels[:, 0]))

    logger.debug(f"RGB signals: R={mean_r}, G={mean_g}, B={mean_b}")
    return (mean_r, mean_g, mean_b)

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
SEQUENCE_LENGTH = 1024
TARGET_FPS = 30
GT_FPS = 60

DATASET_PATH = r"/content/drive/MyDrive/data/dataset"

In [31]:
import scipy.io

SEQUENCE_LENGTH = 1024 # Kept from cell 10

# --- STEP 1: CREATE DUMMY SIGNALS (PLACEHOLDER) ---
# In a real scenario, you would load these 3 signals from your .mat file
# Example: data = scipy.io.loadmat('clean_signals.mat')
#        clean_signals = data['clean_signals']

def create_dummy_signals():
    print("Creating dummy 'clean' and 'noise' data...")
    t = np.linspace(0, 10 * 2 * np.pi, SEQUENCE_LENGTH)

    # 1. 3 'clean' signals (70, 88, 96 bpm)
    # Corresponding frequencies: 70/60=1.17Hz, 88/60=1.47Hz, 96/60=1.6Hz
    clean_70 = np.sin(1.17 * t) + 0.2 * np.sin(2.34 * t)
    clean_88 = np.sin(1.47 * t) + 0.2 * np.sin(2.94 * t)
    clean_96 = np.sin(1.60 * t) + 0.2 * np.sin(3.20 * t)

    # Standardize the 3 clean signals (similar to StandardScaler)
    def standardize(s):
        return (s - np.mean(s)) / np.std(s)

    clean_signals = np.array([
        standardize(clean_70),
        standardize(clean_88),
        standardize(clean_96)
    ])

    # 2. 10 'noise' signals (realistic noise trends)
    # These are low-frequency noise signals (simulating motion/baseline drift)
    noise_signals = []
    for _ in range(10):
        # Create random noise and apply low-pass filtering to simulate a 'trend'
        noise = np.random.randn(SEQUENCE_LENGTH)
        b = np.ones(50) / 50 # Moving average filter
        noise_trend = np.convolve(noise, b, mode='same')
        # Scale the magnitude of the noise randomly
        noise_signals.append(standardize(noise_trend) * np.random.uniform(0.5, 2.0))

    noise_signals = np.array(noise_signals)

    print(f"Shape of dummy 'clean_signals': {clean_signals.shape}")
    print(f"Shape of dummy 'noise_signals': {noise_signals.shape}")

    return clean_signals, noise_signals

# --- STEP 2: DATA AUGMENTATION FUNCTION ---

def load_and_augment_signals(clean_signals, noise_signals, num_augmented_samples=600):
    """
    Generates a large dataset from 3 clean samples and N noise samples.
    Total samples will be: num_augmented_samples + len(clean_signals)
    """

    all_X_noisy = []
    all_Y_clean = []

    num_clean = len(clean_signals)
    num_noise = len(noise_signals)

    print(f"Starting to generate {num_augmented_samples} augmented samples...")

    # Add the 3 original clean samples (with very minor noise)
    for i in range(num_clean):
        all_X_noisy.append(clean_signals[i] + np.random.randn(SEQUENCE_LENGTH) * 0.01)
        all_Y_clean.append(clean_signals[i])

    # Start the augmentation loop
    for _ in tqdm(range(num_augmented_samples), desc="Augmenting"):
        # 1. Select 1 random clean signal
        y_clean = clean_signals[np.random.randint(0, num_clean)]

        # 2. Select 1 random noise trend
        noise_base = noise_signals[np.random.randint(0, num_noise)]

        # 3. Apply Shifting
        # Randomly shift the noise signal left/right
        # (up to 25% of the sequence length)
        max_shift = SEQUENCE_LENGTH // 4
        shift_amount = np.random.randint(-max_shift, max_shift)
        noise_shifted = np.roll(noise_base, shift_amount)

        # 4. Apply Scaling
        # Randomly change the noise intensity
        scale_factor = np.random.uniform(0.3, 3.0) # From 30% to 300%
        noise_processed = noise_shifted * scale_factor

        # 5. Add noise to the clean signal to create X
        x_noisy = y_clean + noise_processed

        # 6. Store the pair (X_noisy, Y_clean)
        all_X_noisy.append(x_noisy)
        all_Y_clean.append(y_clean)

    X_data = np.array(all_X_noisy, dtype=np.float32)
    Y_data = np.array(all_Y_clean, dtype=np.float32)

    print(f"Augmentation complete. Total {len(X_data)} sequences.")
    return X_data, Y_data

In [32]:
def create_lstm_model(seq_length=1024, features=1):
    """
    Creates the LSTM model architecture.
    Input: (None, 1024, 1)
    Output: (None, 1024, 1)
    """
    # The input shape is now (1024, 1)
    inputs = Input(shape=(seq_length, features))

    # 2 stacked LSTM layers
    x = LSTM(units=128, return_sequences=True)(inputs)
    x = Dropout(0.2)(x)
    x = LSTM(units=64, return_sequences=True)(x)
    x = Dropout(0.2)(x)

    # The TimeDistributed layer applies a Dense layer to *each*
    # time-step in the sequence.
    outputs = TimeDistributed(Dense(1, activation='linear'))(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [33]:
if __name__ == "__main__":

    # --- Step 1: Load and Augment Data ---
    # (This code section replaces the load_and_process_dataset function)

    # 1.1. Load (or create) the base clean signals
    # !! REPLACE THIS STEP WITH LOADING YOUR OWN .MAT FILE !!
    clean_signals, noise_signals = create_dummy_signals()

    # 1.2. Run augmentation
    # Generate 600 augmented samples, plus 3 original samples = 603 total sequences
    X_data_raw, Y_data_raw = load_and_augment_signals(
        clean_signals,
        noise_signals,
        num_augmented_samples=600
    )

    # --- Step 2: Data Normalization ---
    # Now that both X and Y are signals, we normalize both

    # 2.1. Normalize X (Input - Noisy signals)
    X_data = np.zeros_like(X_data_raw)
    for i in range(X_data_raw.shape[0]):
        scaler_x = StandardScaler()
        X_data[i, :] = scaler_x.fit_transform(X_data_raw[i, :].reshape(-1, 1)).flatten()

    # 2.2. Normalize Y (Output - Clean signals)
    Y_data = np.zeros_like(Y_data_raw)
    for i in range(Y_data_raw.shape[0]):
        scaler_y = StandardScaler()
        Y_data[i, :] = scaler_y.fit_transform(Y_data_raw[i, :].reshape(-1, 1)).flatten()

    # Add the final dimension to X and Y to match the LSTM input
    # Shape (N, 1024) -> (N, 1024, 1)
    X_data = np.expand_dims(X_data, axis=-1)
    Y_data = np.expand_dims(Y_data, axis=-1)

    # --- Step 3: Split Train/Validation ---
    X_train, X_val, Y_train, Y_val = train_test_split(
        X_data, Y_data, test_size=0.2, random_state=42
    )

    logger.info(f"Kích thước X_train: {X_train.shape}")
    logger.info(f"Kích thước Y_train: {Y_train.shape}")
    logger.info(f"Kích thước X_val: {X_val.shape}")
    logger.info(f"Kích thước Y_val: {Y_val.shape}")

    # --- Step 4: Create and compile model ---
    model = create_lstm_model(seq_length=SEQUENCE_LENGTH, features=1)

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mean_squared_error'
    )

    model.summary()

    # --- Bước 5: Huấn luyện Model --
    EPOCHS = 50
    BATCH_SIZE = 32

    print("Starting training...")

    history = model.fit(
        X_train, Y_train,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val, Y_val),
        callbacks=[
            tf.keras.callbacks.ModelCheckpoint('my_lstm_denoising_model.h5', save_best_only=True, monitor='val_loss'),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        ]
    )

    logger.info("Training complete. Model saved to 'my_lstm_denoising_model.h5'")

Creating dummy 'clean' and 'noise' data...
Shape of dummy 'clean_signals': (3, 1024)
Shape of dummy 'noise_signals': (10, 1024)
Starting to generate 600 augmented samples...


Augmenting: 100%|██████████| 600/600 [00:00<00:00, 8105.59it/s]

Augmentation complete. Total 603 sequences.


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1024, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 1024, 128)      │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1024, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 1024, 64)       │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 1024, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 1024, 1)        │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 220ms/step - loss: 0.4544 - val_loss: 0.4182 - learning_rate: 2.0000e-04
Epoch 40/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 0.4396

16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 204ms/step - loss: 0.4392 - val_loss: 0.4076 - learning_rate: 2.0000e-04
Epoch 41/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step - loss: 0.4446

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 203ms/step - loss: 0.4442 - val_loss: 0.4002 - learning_rate: 2.0000e-04
Epoch 42/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 204ms/step - loss: 0.4061 - val_loss: 0.4249 - learning_rate: 2.0000e-04
Epoch 43/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 228ms/step - loss: 0.4435 - val_loss: 0.4155 - learning_rate: 2.0000e-04
Epoch 44/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 201ms/step - loss: 0.4265 - val_loss: 0.4373 - learning_rate: 2.0000e-04
Epoch 45/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 201ms/step - loss: 0.4217 - val_loss: 0.4019 - learning_rate: 2.0000e-04
Epoch 46/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - loss: 0.4128

16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - loss: 0.4129 - val_loss: 0.3988 - learning_rate: 2.0000e-04
Epoch 47/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.4139

16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 226ms/step - loss: 0.4131 - val_loss: 0.3824 - learning_rate: 2.0000e-04
Epoch 48/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 201ms/step - loss: 0.4654 - val_loss: 0.4308 - learning_rate: 2.0000e-04
Epoch 49/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 202ms/step - loss: 0.4535 - val_loss: 0.4291 - learning_rate: 2.0000e-04
Epoch 50/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 217ms/step - loss: 0.4323 - val_loss: 0.3915 - learning_rate: 2.0000e-04
